### KRX 데이터정보시스템을 이용하여 kospi, kosdaq자료 취득
- yahoo의 경우, 데이터 누락이 가끔 발생하여 의도치 않게 분석 왜곡현상이 발생할 수 있음.

In [1]:
from selenium import webdriver as wd
from selenium.webdriver import ActionChains # scroll down 사용하기 위하여 선서
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as bs

import datetime, time
from datetime import date

import pandas as pd
import numpy as np
import requests
import time
import os, sys

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# !pip install chromedriver_autoinstaller

import chromedriver_autoinstaller

driver_name = chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path
time.sleep(1)

In [4]:
# driver = wd.Chrome(driver_name)
driver = wd.Chrome(service=Service(ChromeDriverManager().install()))

In [5]:
def get_data_kos(kos_id):

    com_ticker = kos_id
    # 회사이름 입력 Q 버튼
   
    css_id = '#jsMdiMenu > div.lnb_tree > ul > li:nth-child(1) > ul > li.sel > div > \
    div.lnb_tree_box > ul > li.sel.on > ul > li.has5dep.sel.on > ul > li.CI-MDI-MENU-NO-CHILD.sel > a'
    driver.find_element(By.CSS_SELECTOR, css_id).click()
    time.sleep(2)
    
    # pop up된 입력창에서 회사이름 입력
    driver.find_element(By.ID, 'tboxindIdx_finder_equidx0_1').clear()         
    time.sleep(1)

    driver.find_element(By.ID, 'tboxindIdx_finder_equidx0_1').send_keys(com_ticker)
    time.sleep(2)
    
    # 검색 버튼 푸시
    driver.find_element(By.CSS_SELECTOR, '#btnindIdx_finder_equidx0_1').click()
    time.sleep(2)

    # 테이블에서 최종 선택
    css_sel = '#jsGrid__finder_equidx0_1 > tbody > tr:nth-child(1) > td.tal.pl20'

    element = WebDriverWait(driver, 30).until(EC.text_to_be_present_in_element((By.CSS_SELECTOR, css_sel), com_ticker))
    # 위 라인은 pop up 창이 사라질 때까지 기다리게 해 줌
    driver.find_element(By.CSS_SELECTOR, css_sel).click()

    return

In [6]:
def set_date(start_date, end_date): # 일정 기간 데이터 취득
    # end_date를 먼저 입력하고 start date 입력. 반대로 하면 start date가 이전날짜로  reset되어짐
    driver.find_element(By.ID, 'endDd').clear()
    driver.find_element(By.ID, 'endDd').send_keys(end_date)
    time.sleep(1)
    driver.find_element(By.ID, 'strtDd').clear()
    driver.find_element(By.ID, 'strtDd').send_keys(start_date)
    time.sleep(1)

In [7]:
def get_data_date(start_date, end_date):
    
#     column_name = ['date', 'close', 'change', 'close_cr', 'open', 'high', 'low', 
#                   'vol', 'vol_amount','total_amount', 'total_counts' ]
    column_name = ['date', 'close', 't1', 'close_cr',  'open', 'high', 'low', 'volume', 't2', 't3']
    # ['일자', '종가', '대비', '등락률', '시가', '고가', '저가', '거래량', 
    #                                 '거래대금', '시가총액', '상장주식수']
 
    start_str = start_date.strftime('%Y-%m-%d')
    ed_str = end_date.strftime('%Y-%m-%d')
    set_date(start_str, ed_str)

    # id가 form별로 중복 구성되어 있어서 아래과 같이 수정함.
    driver.find_element(By.XPATH, '(.//*[@id="jsSearchButton"])[2]').click()
    time.sleep(5)

    df = pd.read_html(driver.page_source, 
                          attrs={"class": "CI-GRID-BODY-TABLE"}, flavor=["lxml", "bs4"])[0]
    df.columns = column_name
    df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y/%m/%d"))
    df_get = df[['date', 'close', 'open', 'high', 'low', 'volume', 'close_cr']]
    
    return df_get

In [8]:
def non_empty_index_df(df_input, start_date, end_date): # 토,일,공휴일등 거래가 없는 일자도 모두 포함
    date_range_ts = pd.date_range(start=start_date, end=end_date)
    df_input.set_index('date', inplace=True)
    df_out = pd.DataFrame(columns = df_input.columns)
    df_out.insert(0, 'date', date_range_ts)
    df_out.set_index('date', inplace=True)
    df_out.update(df_input)
    df_out.reset_index(inplace=True)
    return df_out

In [9]:
def concat_df(df_o, df):
    df_o = pd.concat([df_o, df], ignore_index=True)
    df_o.drop_duplicates(subset=['date'], keep='last', inplace=True)
#     df_o.drop_duplicates(subset=['date'], keep='first', inplace=True)
    df_o.sort_values(by=[df_o.columns[0]], inplace=True)
    df_o.index = np.arange(0, len(df_o))  # 일련 번호 오름차순으로 재 설정
    return df_o

In [10]:
def make_pickle(df, pkl_name):
    pkl_directory = 'data/common_pkl/'
    try:
        if not os.path.exists(pkl_directory):
            os.makedirs(pkl_directory)
    except OSError:
        print("Error: Failed to create the directory.")

    # 데이터 저장: ../data/spx.pkl
    df.to_pickle(pkl_directory+pkl_name)
    df.to_csv(pkl_directory+pkl_name.replace('pkl','csv'))

In [11]:
def get_ticker_data(ticker, startdate, enddate, col_name):
    ydata = yf.Ticker(ticker)
    rdata = ydata.history(period="2y") # 오늘부터 2년치
    rdata.reset_index('Date', inplace=True)
    rdata['Date'] = rdata['Date'].dt.date  # datetime64 to datetime.date()
    rdata = rdata[(rdata['Date'] <= enddate) & (rdata['Date'] >= startdate)] 
    rdata['temp'] = rdata['Close'].shift(1)
    val_temp = (rdata['Close'] - rdata['temp'])/rdata['temp']*100
    rdata[f'{col_name}_cr'] = val_temp.map("{:.2f}%".format)
    rdata = rdata[['Date', 'Close', 'Open', 'High', 'Low', 'Volume', f'{col_name}_cr']] # 필요한 column만 남김
    rdata.columns = ['date', f'{col_name}', 'open', 'high', 'low', 'volume', f'{col_name}_cr'] # column이름 통일
    rdata.reset_index(drop=True, inplace=True) # index번호를 0부터 재정리
    return rdata

In [12]:
# driver.set_window_position(-10000,0) # hide windows
main_url = 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201010105'
# 개별종목 시세추이 data-menu-id: MDC0201020103
driver.get(main_url)
time.sleep(1)

In [13]:
# select 개별지수종합정보 side menu
data_menu_id = '#jsMdiMenu > div.lnb_tree > ul > li:nth-child(1) > ul > li.sel > div > div.lnb_tree_box > ul > li.sel > ul > li.has5dep.sel.on > ul > li.CI-MDI-MENU-NO-CHILD.sel > a'
driver.find_element(By.CSS_SELECTOR, data_menu_id).click()
time.sleep(1)

In [14]:
# 일자별 화면으로 이동. 이동후부터 element id 내용이 ------0_1로 변경되어 진행되므로, 이 과정은 꼭 거쳐야 함.
driver.find_element(By.ID, 'jsOpenView_1').click()
time.sleep(1)

In [15]:
# # 백만원 단위 표시 선정
# css_sel = '#MDCSTAT017_FORM > div.CI-MDI-UNIT-WRAP > div > p:nth-child(2) \
#            > select.CI-MDI-UNIT-MONEY > option:nth-child(3)'
# # driver.find_element_by_css_selector(css_sel).click()  # by_scc 형식이 없어짐.
# driver.find_element(By.CSS_SELECTOR, css_sel).click()
# time.sleep(1)

In [16]:
code = {'kospi':'코스피', 'kosdaq':'코스닥'}
# code = {'kosdaq':'코스닥'}

In [17]:
pkl_directory = 'data/common_pkl/'
for com, val in code.items():
    com_name = f'{val}'
    pkl_name= f'{com}.pkl'
    try :
        df_o = pd.read_pickle(pkl_directory + pkl_name)
        start_date = df_o['date'].iloc[len(df_o)-1]
    except :
        start_date = datetime.date(2022, 1, 1)   # 데이터 취득 시작 일자     

#     start_date = datetime.date(2023, 5, 11)  # 데이터 취득 오류시 일시 사용
        
    end_date = datetime.date.today()
    get_data_kos(com_name)
    df_get = get_data_date(start_date, end_date)
    df_out = non_empty_index_df(df_get, start_date, end_date)
    df_out['date'] = df_out['date'].dt.date # convert to datetime
    df_out.columns = ['date', f'{com}', 'open', 'high', 'low', 'volume', f'{com}_cr'] # column이름 통일
    try :
        df_o = concat_df(df_o, df_out) # append df to original df
    except :
        df_o = df_out.copy()
        
    df_o.replace(np.nan, '', inplace=True)
    df_o.to_pickle(pkl_directory+pkl_name)
    df_o.to_csv(pkl_directory+pkl_name.replace('pkl','csv'))

In [18]:
driver.close()
driver.quit()

In [19]:
# # 최초 데이터 취득시 사용됨.

# pkl_directory = 'data/common_pkl/'
# df = pd.read_csv(pkl_directory + 'kospi1.csv', encoding='cp949' )
# columns = ['date', 'kospi', 't1', 'kospi_cr',  'open', 'high', 'low', 'volume', 't2', 't3']
# df.columns = columns
# df['date'] = pd.to_datetime(df['date']).dt.date
# df_o = df[['date', 'kospi', 'open', 'high', 'low', 'volume', 'kospi_cr']]
# df_o = df_o.sort_values(by='date')
# df_o.index = np.arange(0,len(df_o))
# make_pickle(df_o, 'kospi.pkl')

In [20]:
# # 최초 데이터 취득시 사용됨.

# pkl_directory = 'data/common_pkl/'
# df = pd.read_csv(pkl_directory + 'kosdaq1.csv', encoding='cp949' )
# columns = ['date', 'kosdaq', 't1', 'kosdaq_cr',  'open', 'high', 'low', 'volume', 't2', 't3']
# df.columns = columns
# df['date'] = pd.to_datetime(df['date']).dt.date
# df_o = df[['date', 'kosdaq', 'open', 'high', 'low', 'volume', 'kosdaq_cr']]
# df_o = df_o.sort_values(by='date')
# df_o.index = np.arange(0,len(df_o))
# make_pickle(df_o, 'kosdaq.pkl')